In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim import Adam
from torchvision import datasets
from torchvision import models
from torchvision.models import DenseNet169_Weights

import matplotlib.pyplot as plt
import random

import numpy as np
from colorama import Fore, Style

In [ ]:
# Results libraries import
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay

In [ ]:
# colorama
red = Fore.RED
green = Fore.GREEN
blue = Fore.BLUE
yellow = Fore.YELLOW
cyan = Fore.CYAN

reset = Style.RESET_ALL

In [ ]:
# Data
d = ".../Rocks/"

loc = "PyDL_C"

# Sub-Categorized data
train_dir = d + loc + "/data/train"
test_dir = d + loc + "/data/test"
valid_dir = d + loc + "/data/validation"

In [ ]:
# Setting the seed
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

print(f'{blue}Global seed set to : {yellow}{seed}\n')

In [ ]:
# Image dimentions
img_dimen = (256, 256)
bs = 16
max_acc_ac = 0
y_pred = []
y_true = []

In [ ]:
# mean & std
# preprocessing | get the data mean and std for normalization
transform = transforms.Compose([
    transforms.Resize(img_dimen),
    transforms.ToTensor()
])

calc_ms = datasets.ImageFolder(root=train_dir, transform=transform)
loader_ms = torch.utils.data.DataLoader(dataset=calc_ms, batch_size=bs, shuffle=False)

mean_calc = 0
std_calc = 0
total_images = 0

for images, _ in loader_ms:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean_calc += images.mean(2).sum(0)
    std_calc += images.std(2).sum(0)
    total_images += batch_samples

mean_calc /= total_images
std_calc /= total_images

print(f'{blue}mean: {yellow}{mean_calc}')
print(f'{blue}std: {yellow}{std_calc}{reset}')

In [ ]:
# org mean & std values
#mean_calc = [0.485, 0.456, 0.406]
#std_calc = [0.229, 0.224, 0.255]

In [ ]:
# Data transformations training set
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(img_dimen),
    transforms.ToTensor(),
    transforms.Normalize(mean_calc, std_calc)
])

# Data transformations for validation and test sets
transform_common = transforms.Compose([
    transforms.Resize(img_dimen),
    transforms.ToTensor(),
    transforms.Normalize(mean_calc, std_calc)
])

# Image dataset
dataset_train = datasets.ImageFolder(root=train_dir, transform=transform_train)
dataset_test = datasets.ImageFolder(root=test_dir, transform=transform_common)
dataset_valid = datasets.ImageFolder(root=valid_dir, transform=transform_common)

In [ ]:
# Hyperparameters
max_epoch = 50
batch_size = bs
learningRate = 0.0001
WeightDecay = 1e-08

# All Information
print(f'{blue}Epochs: {yellow}{max_epoch}{reset}')
print(f'{blue}Batch size: {yellow}{batch_size}{reset}')
print(f'{blue}Learning rate: {yellow}{learningRate}{reset}')
print(f'{blue}Weight decay: {yellow}{WeightDecay}{reset}')

In [ ]:
# Dataloaders
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)

In [ ]:
# DenseNet
model = models.densenet169(weights=DenseNet169_Weights.IMAGENET1K_V1)

num_classes = len(dataset_train.classes)

# Freeze the model parameters
#for param in model.parameters():
#    param.requires_grad = False

# change the last layer of the model
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(in_features=num_ftrs, out_features=num_classes)

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learningRate, weight_decay=WeightDecay)
print(f'{blue}Device: {yellow}{device}{reset}')

In [ ]:
# TRAINING

# Loss metrics
train_loss = []
val_loss = []
# Accuracy metrics
train_acc = []
val_acc = []
# best model score
max_score = 0

for epoch in range(max_epoch):
    model.train()

    # Metrics initialization
    running_loss = 0.0
    num_correct = 0

    # TRAINING
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Predictions | forward pass | OUTPUT
        outputs = model(inputs)
        # Loss | backward pass | GRADIENT
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        # Count correct predictions
        num_correct += (predicted == labels).sum().item()

    # ---------------------------------------------------------------------------
    # Training loss
    train_lss = running_loss / len(train_loader)
    train_loss.append(train_lss)

    # Training accuracy
    train_accuracy = 100 * num_correct / len(train_loader.dataset)
    train_acc.append(train_accuracy)
    # ---------------------------------------------------------------------------

    model.eval()
    correct = 0
    valid_loss = 0

    # VALIDATION
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Predictions
            outputs = model(inputs)
            # Count correct predictions
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            # Loss
            valid_loss += criterion(outputs, labels).item()

    # --------------------------------------------------------------------------
    #Validation loss
    val_lss = valid_loss / len(valid_loader)
    val_loss.append(val_lss)

    # Validation accuracy
    val_accuracy = 100 * correct / len(valid_loader.dataset)
    val_acc.append(val_accuracy)
    # --------------------------------------------------------------------------

    print(f'{cyan}\nEPOCH {epoch + 1}{reset}')
    print(f"Loss: {red}{train_lss}{reset}, Validation Accuracy: {red}{val_accuracy}%{reset}, Training Accuracy: {red}{train_accuracy}%")

    # Save the best model
    if val_accuracy > max_score:
        max_score = val_accuracy
        path = d + loc + '/models/DenseNet_169_T.pth'
        torch.save(model.state_dict(), path)
        print(f'{green}Improvement! Model saved!{reset}')

print(f'{yellow}Training finished!\n')

# Save the Final model
path = d + loc + '/models/DenseNet_169_F.pth'
torch.save(model.state_dict(), path)

In [ ]:
# Graph of training and validation: loss and accuracy | dual plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Loss plot
ax1.set_title("Loss")
ax1.plot(val_loss, color='red', label='Validation loss', linestyle='dashed')
ax1.plot(train_loss, color='orange', label='Training loss')
ax1.legend()
ax1.set_xlabel("Iterations")
ax1.set_ylabel("Loss")

# Accuracy plot
ax2.set_title("Accuracy")
ax2.plot(val_acc, color='red', label='Validation accuracy', linestyle='dashed')
ax2.plot(train_acc, color='orange', label='Training accuracy')
ax2.legend()
ax2.set_xlabel("Iterations")
ax2.set_ylabel("Accuracy")

plt.show()

In [ ]:
# TESTING on FINAL Model
acc_final = 0
correct = 0
total = 0

y_pred_F = []
y_true_F = []

model.eval()

with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Predictions | forward pass | OUTPUT
        outputs = model(inputs)
        # Count correct predictions
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        y_pred_F.extend(predicted.tolist())
        y_true_F.extend(labels.tolist())

acc_final = 100 * correct / total
print(f"{blue}Test Accuracy (Final Model): {red}{100 * correct / total}%")

In [ ]:
# TESTING on BEST Model
b_model = models.densenet169(weights=DenseNet169_Weights.IMAGENET1K_V1)
num_ftrs = b_model.classifier.in_features
b_model.classifier = nn.Linear(num_ftrs, num_classes)
b_model.load_state_dict(torch.load(d + loc + '/models/DenseNet_169_T.pth'))

acc_best = 0
correct = 0
total = 0

y_pred_B = []
y_true_B = []

b_model.eval()
b_model.to(device)

with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Predictions | forward pass | OUTPUT
        outputs = b_model(inputs)
        # Count correct predictions
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        y_pred_B.extend(predicted.tolist())
        y_true_B.extend(labels.tolist())

acc_best = 100 * correct / total
print(f"{blue}Test Accuracy (Best Model): {red}{100 * correct / total}%")

In [ ]:
if acc_final > acc_best:
    max_acc_ac = acc_final
    y_pred = y_pred_F
    y_true = y_true_F
    print(f"{blue}Best Accuracy on Final Model! {red}{max_acc_ac}{reset}")
    
else:
    model = b_model
    max_acc_ac = acc_best
    y_pred = y_pred_B
    y_true = y_true_B
    print(f"{blue}Best Accuracy on Highest Accuracy Validation Model! {red}{max_acc_ac}{reset}")

In [ ]:
# Classification Report
print(f"{blue}Classification Report:")
print(classification_report(y_true, y_pred, target_names=dataset_test.classes))

In [ ]:
# F-1 Score
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"{blue}F-1 Score: {red}{f1 * 100}%{reset}")

In [ ]:
# Precision
precision = precision_score(y_true, y_pred, average='weighted')
print(f"{blue}Precision: {red}{precision * 100}%{reset}")

In [ ]:
# Recall | Sensitivity
recall = recall_score(y_true, y_pred, average='weighted')
print(f"{blue}Recall: {red}{recall * 100}%{reset}")

In [ ]:
def tp_calc (y_true, y_pred, class_label):
    tp = 0
    for i in range(len(y_true)):
        if y_true[i] == class_label and y_pred[i] == class_label:
            tp += 1
    return tp
    
def tn_calc (y_true, y_pred, class_label):
    tn = 0
    for i in range(len(y_true)):
        if y_true[i] != class_label and y_pred[i] != class_label:
            tn += 1
    return tn
    
def fp_calc (y_true, y_pred, class_label):
    fp = 0
    for i in range(len(y_true)):
        if y_true[i] != class_label and y_pred[i] == class_label:
            fp += 1
    return fp
    
def fn_calc (y_true, y_pred, class_label):
    fn = 0
    for i in range(len(y_true)):
        if y_true[i] == class_label and y_pred[i] != class_label:
            fn += 1
    return fn

In [ ]:
def calculate_specificity(y_true, y_pred, class_index):
    # Convert y_true and y_pred to numpy arrays if they are lists
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Identify true positive, false positive, true negative, and false negative counts
    # true_positive = np.sum((y_true == class_index) & (y_pred == class_index))
    false_positive = np.sum((y_true != class_index) & (y_pred == class_index))
    true_negative = np.sum((y_true != class_index) & (y_pred != class_index))
    # false_negative = np.sum((y_true == class_index) & (y_pred != class_index))

    # Calculate specificity
    specificity = true_negative / (true_negative + false_positive)

    return specificity

def calculate_multi_class_specificity(y_true, y_pred):
    num_classes = len(np.unique(y_true))
    specificity_scores = []

    for class_index in range(num_classes):
        specificity = calculate_specificity(y_true, y_pred, class_index)
        specificity_scores.append(specificity)

    # Calculate the average specificity across all classes
    average_specificity = np.mean(specificity_scores)

    return average_specificity, specificity_scores


# Calculate the specificity
average_specificity, specificity_scores = calculate_multi_class_specificity(y_true, y_pred)
print(f"{blue}Specificity: {red}{average_specificity * 100}%{reset}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dataset_test.classes)
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title("DenseNet-169")
plt.show()